In [300]:
import pandas as pd

In [301]:
df = pd.read_csv("final_data/final_dataset.csv", index_col=0)
df = df.sort_values("date")
df = df.reset_index(drop=True)
del df["home"]
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]
del df["home_opp"]


In [302]:
df_bet = pd.concat([df["team"], df["team_opp"], df["season"], df["date"], df["HomeML"], df["AwayML"]], axis=1)

In [303]:
del df["AwayTeam"]
del df["HomeSpread_AtOpen"]
del df["HomeSpread_AtClose"]
del df["Over_AtOpen"]
del df["Over_AtClose"]
del df["HomeML"]
del df["AwayML"]
del df["2H_HomeSpread"]
del df["2H_Over"]

In [304]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]
valid_columns = df.columns[~df.columns.isin(nulls.index)]
valid_columns
df = df[valid_columns].copy()
df["won"] = df["won"].astype(int, errors="ignore")

In [305]:
home_stats_list = []
away_stats_list = []
start = False
for k in df.keys():
    if k == "team": break
    if k == "mp":
        start = True
    if start:
        home_stats_list.append(k)
start = False
for k in df.keys():
    if k == "team_opp": break
    if k == "mp_opp":
        start = True
    if start:
        away_stats_list.append(k)
home_stats_list.append("total")
away_stats_list.append("total_opp")
home_stats_list.append("won")
away_stats_list.append("won_opp")

new_keys = ["team", "team_opp", "season", "date"] + home_stats_list + away_stats_list + ["target"]


In [306]:
def get_season_avg(data: pd.DataFrame):
    teams_list = data["team"].unique()
    df_season = pd.DataFrame(columns=new_keys)
    teams_num = {k: [0] * len(home_stats_list) for k in teams_list}
    teams_denom = {k: 0 for k in teams_list}

    for i in data.index:
        team1 = data.loc[i, "team"]
        team2 = data.loc[i, "team_opp"]
        new_row = [team1, team2, data.loc[i, "season"], data.loc[i, "date"]]

        temp1 = [num / teams_denom[team1] if teams_denom[team1] > 0 else float('nan') for num in teams_num[team1]]
        temp2 = [num / teams_denom[team2] if teams_denom[team2] > 0 else float('nan') for num in teams_num[team2]]

        new_row.extend(temp1)
        new_row.extend(temp2)
        new_row.append(data.loc[i, "won"])

        for idx, stat in enumerate(home_stats_list):
            teams_num[team1][idx] += data.loc[i, stat]
        for idx, stat in enumerate(away_stats_list):
            if stat == "won_opp":
                teams_num[team2][idx] += (data.loc[i, "won"] + 1) % 2
            else:
                teams_num[team2][idx] += data.loc[i, stat]

        teams_denom[team1] += 1
        teams_denom[team2] += 1

        # Append to dataframe
        df_season.loc[len(df_season)] = new_row

    return df_season




In [307]:
df_avg = pd.DataFrame(columns=new_keys)
seasons_list = df["season"].unique()
for s in seasons_list:
    temp = get_season_avg(df[df["season"] == s])
    df_avg = pd.concat([df_avg, temp], ignore_index=True)


df_avg.to_csv("final_data/averages.csv")
df_bet.to_csv("final_data/betting.csv")

C:\Users\alanh\AppData\Local\Temp\ipykernel_18172\1626689790.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_avg = pd.concat([df_avg, temp], ignore_index=True)
